# Momentum Trading

### The Strategy
- ##### Check the past 12 month performance of all index stocks
- ##### Buy the top performers and hold them for 1 month

### Accounting for Survivorship Bias
- #### Ensure that stocks are added / removed in line with historical data

## Imports

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime

## Load Data

In [2]:
start = "2015-01-01"
end = datetime.today().strftime('%Y-%m-%d')
start, end

('2015-01-01', '2025-04-06')

In [58]:
# Get up to date S&P500 stock information from it's wikipedia page
wiki = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")

In [59]:
# Retrieve the current S&P500 stock list
current = wiki[0].copy()
current.sort_values('Date added', ascending=False).head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
151,DASH,DoorDash,Consumer Discretionary,Specialized Consumer Services,"San Francisco, California",2025-03-24,1792789,2012
446,TKO,TKO Group Holdings,Communication Services,Movies & Entertainment,"New York City, New York",2025-03-24,1973266,2023
492,WSM,"Williams-Sonoma, Inc.",Consumer Discretionary,Homefurnishing Retail,"San Francisco, California",2025-03-24,719955,1956
182,EXE,Expand Energy,Energy,Oil & Gas Exploration & Production,"Oklahoma City, Oklahoma",2025-03-24,895126,1989
495,WDAY,"Workday, Inc.",Information Technology,Application Software,"Pleasanton, California",2024-12-23,1327811,2005


In [60]:
current.set_index(pd.to_datetime(current['Date added']), inplace=True)
current.sort_index(inplace=True)
current

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
Date added,,,,,,,,
1957-03-04,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1957-03-04,BMY,Bristol Myers Squibb,Health Care,Pharmaceuticals,"New York City, New York",1957-03-04,14272,1989 (1887)
1957-03-04,HSY,Hershey Company (The),Consumer Staples,Packaged Foods & Meats,"Hershey, Pennsylvania",1957-03-04,47111,1894
1957-03-04,HIG,Hartford (The),Financials,Property & Casualty Insurance,"Hartford, Connecticut",1957-03-04,874766,1810
1957-03-04,HAL,Halliburton,Energy,Oil & Gas Equipment & Services,"Houston, Texas",1957-03-04,45012,1919
...,...,...,...,...,...,...,...,...
2024-12-23,WDAY,"Workday, Inc.",Information Technology,Application Software,"Pleasanton, California",2024-12-23,1327811,2005
2025-03-24,TKO,TKO Group Holdings,Communication Services,Movies & Entertainment,"New York City, New York",2025-03-24,1973266,2023
2025-03-24,DASH,DoorDash,Consumer Discretionary,Specialized Consumer Services,"San Francisco, California",2025-03-24,1792789,2012


In [61]:
# Add current stocks to the list of complete stocks we need to load from yfinance
stocks = current.Symbol.to_list()
stocks

['MMM',
 'BMY',
 'HSY',
 'HIG',
 'HAL',
 'GIS',
 'GD',
 'GE',
 'F',
 'XOM',
 'ADM',
 'EXC',
 'ETR',
 'IBM',
 'EIX',
 'DTE',
 'DE',
 'CVS',
 'CSX',
 'ED',
 'COP',
 'CL',
 'KO',
 'CMS',
 'CVX',
 'CAT',
 'CPB',
 'ETN',
 'IP',
 'HON',
 'PFE',
 'XEL',
 'ABT',
 'UNP',
 'SO',
 'SLB',
 'SPGI',
 'RTX',
 'PEG',
 'PG',
 'PPG',
 'PEP',
 'OXY',
 'NOC',
 'BA',
 'MSI',
 'KMB',
 'MRK',
 'MO',
 'LMT',
 'NSC',
 'AEP',
 'KR',
 'SHW',
 'EMR',
 'CMI',
 'CLX',
 'NEM',
 'MCD',
 'LLY',
 'BAX',
 'BDX',
 'JNJ',
 'GPC',
 'HPQ',
 'WMB',
 'JPM',
 'IFF',
 'NEE',
 'DIS',
 'CI',
 'TAP',
 'BAC',
 'DUK',
 'WFC',
 'AXP',
 'INTC',
 'TGT',
 'TXT',
 'WY',
 'WBA',
 'AIG',
 'PCAR',
 'FDX',
 'ADP',
 'MAS',
 'GWW',
 'WMT',
 'SNA',
 'SWK',
 'BF.B',
 'AAPL',
 'CAG',
 'VZ',
 'T',
 'LOW',
 'PHM',
 'HES',
 'HAS',
 'BALL',
 'APD',
 'NUE',
 'RVTY',
 'CNP',
 'TJX',
 'DOV',
 'PH',
 'ITW',
 'MDT',
 'SYY',
 'MMC',
 'AVY',
 'HD',
 'PNC',
 'C',
 'NKE',
 'ECL',
 'GL',
 'ORCL',
 'K',
 'ADSK',
 'AEE',
 'AMGN',
 'LIN',
 'IPG',
 'MS',
 'COST',


In [62]:
# Stocks that were added after our start point will need to be filtered to account for survivorship bias
added = current[current.index >= start].copy()
added

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
Date added,,,,,,,,
2015-01-27,HCA,HCA Healthcare,Health Care,Health Care Facilities,"Nashville, Tennessee",2015-01-27,860730,1968
2015-03-12,SWKS,Skyworks Solutions,Information Technology,Semiconductors,"Irvine, California",2015-03-12,4127,2002
2015-03-17,HSIC,Henry Schein,Health Care,Health Care Distributors,"Melville, New York",2015-03-17,1000228,1932
2015-03-20,EQIX,Equinix,Real Estate,Data Center REITs,"Redwood City, California",2015-03-20,1101239,1998
2015-04-07,O,Realty Income,Real Estate,Retail REITs,"San Diego, California",2015-04-07,726728,1969
...,...,...,...,...,...,...,...,...
2024-12-23,WDAY,"Workday, Inc.",Information Technology,Application Software,"Pleasanton, California",2024-12-23,1327811,2005
2025-03-24,TKO,TKO Group Holdings,Communication Services,Movies & Entertainment,"New York City, New York",2025-03-24,1973266,2023
2025-03-24,DASH,DoorDash,Consumer Discretionary,Specialized Consumer Services,"San Francisco, California",2025-03-24,1792789,2012


In [63]:
# Retrieve a list of stocks that are not currently in the S&P500 stock list
removed = wiki[1][['Date', 'Removed']].copy()
removed.head()

Date Removed                 
                Date  Ticker         Security
0     March 24, 2025     BWA       BorgWarner
1     March 24, 2025     TFX         Teleflex
2     March 24, 2025      CE         Celanese
3     March 24, 2025     FMC  FMC Corporation
4  December 23, 2024    QRVO            Qorvo

In [64]:
# Stocks that were removed after our start point will need to be included until their removal date in order to account for survivorship bias
removed.set_index(pd.to_datetime(removed.Date.Date), inplace=True)
removed.sort_index(inplace=True)
removed

Date Removed                 
                         Date  Ticker         Security
Date                                                  
1997-06-17      June 17, 1997     USL           USLife
1998-12-11  December 11, 1998      AN            Amoco
1998-12-11  December 11, 1998     SUN       SunAmerica
1998-12-11  December 11, 1998     GRN       General Re
1999-04-12     April 12, 1999     NaN              NaN
...                       ...     ...              ...
2024-12-23  December 23, 2024    QRVO            Qorvo
2025-03-24     March 24, 2025     FMC  FMC Corporation
2025-03-24     March 24, 2025      CE         Celanese
2025-03-24     March 24, 2025     TFX         Teleflex
2025-03-24     March 24, 2025     BWA       BorgWarner

[368 rows x 3 columns]

In [65]:
removed = removed[removed.index >= start].Removed
removed

,Ticker,Security
Date,,
2015-01-27,COV,Covidien
2015-01-27,SWY,Safeway
2015-03-12,PETM,PetSmart
2015-03-18,CFN,Carefusion
2015-03-23,AGN,Allergan
...,...,...
2024-12-23,QRVO,Qorvo
2025-03-24,FMC,FMC Corporation
2025-03-24,CE,Celanese


In [19]:
# I will drop values we don't have ticker information for
removed[removed.Ticker.isna()]

,Ticker,Security
Date,,
2024-09-30,NaN,NaN
2024-04-02,NaN,NaN
2024-04-01,NaN,NaN
2023-10-02,NaN,NaN
2023-01-04,NaN,NaN
2022-12-15,NaN,NaN
2022-02-02,NaN,NaN
2021-06-03,NaN,NaN
2020-10-09,NaN,NaN


In [66]:
removed.dropna(inplace=True)
removed

,Ticker,Security
Date,,
2015-01-27,COV,Covidien
2015-01-27,SWY,Safeway
2015-03-12,PETM,PetSmart
2015-03-18,CFN,Carefusion
2015-03-23,AGN,Allergan
...,...,...
2024-12-23,QRVO,Qorvo
2025-03-24,FMC,FMC Corporation
2025-03-24,CE,Celanese


In [67]:
removed.isna().any().any()

np.False_

In [68]:
# We will also need to retrieve the removed stocks from yfinance
stocks.extend(removed.Ticker.to_list())
len(stocks)

723

In [71]:
len(set(stocks))

715

## Download Stock Data From Yahoo Finance

In [72]:
# Load all relevent stocks from yfinance
df = yf.download(stocks, start=start, end=end)['Close']

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  715 of 715 completed

101 Failed downloads:
['CERN', 'FBHS', 'KSU', 'VAR', 'WIN', 'FRC', 'NBL', 'DISCK', 'HFC', 'APC', 'PXD', 'DTV', 'NLSN', 'CELG', 'BBBY', 'MON', 'ABMD', 'ALXN', 'CXO', 'ENDP', 'TSS', 'XL', 'ATVI', 'YHOO', 'XEC', 'QEP', 'CTXS', 'ADS', 'FTR', 'DISH', 'DWDP', 'MXIM', 'ARNC', 'ETFC', 'DO', 'RTN', 'BRK.B', 'SIVB', 'AGN', 'TWTR', 'DISCA', 'VIAB', 'GPS', 'DRE', 'RHT', 'ESV', 'WCG', 'MNK', 'AVP', 'CHK', 'LM', 'SWN', 'DNR', 'XLNX', 'FLIR', 'PBCT', 'TIF', 'LLL']: YFTzMissingError('possibly delisted; no timezone found')
['MJN', 'HSP', 'SIAL', 'BF.B', 'WYN', 'TWC', 'GMCR', 'LO', 'BCR', 'LVLT', 'LEN', 'BRCM', 'LLTC', 'RAI', 'SPLS', 'TYC', 'KRFT', 'CMCSK', 'GGP', 'HCBK', 'STJ', 'SWY', 'POM', 'SNI', 'CFN', 'DPS', 'WFM', 'BXLT', 'FDO', 'JOY', 'ARG', 'CVC', 'CAM', 'CPGX']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2025-04-06)')
['AET', 'TWX', 'BMS', 'CSRA', 'SCG', 'HOT', 'ANDV']: ValueError('Len

In [76]:
df

Ticker,A,AA,AAL,AAP,AAPL,ABBV,ABMD,ABNB,ABT,ACE,...,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,37.273823,35.912807,51.079914,141.844131,24.320438,43.156208,NaN,NaN,36.915031,NaN,...,59.510231,46.345871,19.412529,33.441803,NaN,42.833298,100.714760,77.430000,21.878542,40.157536
2015-01-05,36.575386,33.832211,51.051487,139.974518,23.635283,42.344051,NaN,NaN,36.923233,NaN,...,57.881908,46.024578,18.988977,31.360487,NaN,41.963024,104.471764,76.339996,21.058773,39.916458
2015-01-06,36.005630,34.080982,50.255596,139.876099,23.637514,42.134453,NaN,NaN,36.503952,NaN,...,57.574200,45.453403,18.734856,31.176060,NaN,41.447956,103.586182,75.790001,20.254478,39.527031
2015-01-07,36.483501,34.962975,50.227161,142.881866,23.968960,43.837383,NaN,NaN,36.799927,NaN,...,58.157578,46.631454,18.974867,31.421959,NaN,42.821465,106.162430,77.720001,20.447815,40.342964
2015-01-08,37.577080,35.958031,50.843040,144.134262,24.889906,44.295864,NaN,NaN,37.556309,NaN,...,59.125603,47.782749,19.497240,31.659069,NaN,43.567413,107.289520,79.379997,20.733965,40.964191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-31,116.732002,30.500000,10.550000,39.209999,222.130005,209.520004,NaN,119.459999,132.649994,NaN,...,118.930000,14.940000,4.830000,119.459999,NaN,157.360001,113.180000,282.559998,49.860001,164.649994
2025-04-01,114.050003,30.139999,10.300000,39.020000,223.190002,206.270004,NaN,121.180000,131.690002,NaN,...,119.040001,14.650000,4.920000,119.879997,NaN,158.919998,112.370003,283.070007,49.480000,161.910004
2025-04-02,115.989998,30.840000,10.590000,39.320000,223.889999,205.190002,NaN,122.800003,131.929993,NaN,...,118.669998,14.870000,4.910000,119.050003,NaN,158.229996,112.500000,287.570007,50.439999,163.500000


In [74]:
df.columns[df.isna().all()]

Index(['ABMD', 'ADS', 'AET', 'AGN', 'ALXN', 'ANDV', 'APC', 'ARG', 'ARNC',
       'ATVI',
       ...
       'VAR', 'VIAB', 'WCG', 'WFM', 'WIN', 'WYN', 'XEC', 'XL', 'XLNX', 'YHOO'],
      dtype='object', name='Ticker', length=101)

In [75]:
df['ACE'][df['ACE'].notna()]

Date
2018-08-28     1.46
2018-08-29     1.46
2018-08-30     1.46
2018-08-31     1.43
2018-09-04     1.43
              ...  
2019-06-03     1.31
2019-06-04     1.33
2019-06-06     1.38
2019-06-07     1.31
2020-01-24    33.00
Name: ACE, Length: 195, dtype: float64

In [77]:
# Save the overall dataset for easy access
df.to_csv('data/sp500_historic.csv')

In [79]:
# Load from csv if YFinance is down
df = pd.read_csv('data/sp500_historic.csv', index_col=0, date_format = pd.to_datetime)
df.index = pd.to_datetime(df.index)
df

,A,AA,AAL,AAP,AAPL,ABBV,ABMD,ABNB,ABT,ACE,...,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,37.273823,35.912807,51.079914,141.844131,24.320438,43.156208,NaN,NaN,36.915031,NaN,...,59.510231,46.345871,19.412529,33.441803,NaN,42.833298,100.714760,77.430000,21.878542,40.157536
2015-01-05,36.575386,33.832211,51.051487,139.974518,23.635283,42.344051,NaN,NaN,36.923233,NaN,...,57.881908,46.024578,18.988977,31.360487,NaN,41.963024,104.471764,76.339996,21.058773,39.916458
2015-01-06,36.005630,34.080982,50.255596,139.876099,23.637514,42.134453,NaN,NaN,36.503952,NaN,...,57.574200,45.453403,18.734856,31.176060,NaN,41.447956,103.586182,75.790001,20.254478,39.527031
2015-01-07,36.483501,34.962975,50.227161,142.881866,23.968960,43.837383,NaN,NaN,36.799927,NaN,...,58.157578,46.631454,18.974867,31.421959,NaN,42.821465,106.162430,77.720001,20.447815,40.342964
2015-01-08,37.577080,35.958031,50.843040,144.134262,24.889906,44.295864,NaN,NaN,37.556309,NaN,...,59.125603,47.782749,19.497240,31.659069,NaN,43.567413,107.289520,79.379997,20.733965,40.964191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-31,116.732002,30.500000,10.550000,39.209999,222.130005,209.520004,NaN,119.459999,132.649994,NaN,...,118.930000,14.940000,4.830000,119.459999,NaN,157.360001,113.180000,282.559998,49.860001,164.649994
2025-04-01,114.050003,30.139999,10.300000,39.020000,223.190002,206.270004,NaN,121.180000,131.690002,NaN,...,119.040001,14.650000,4.920000,119.879997,NaN,158.919998,112.370003,283.070007,49.480000,161.910004
2025-04-02,115.989998,30.840000,10.590000,39.320000,223.889999,205.190002,NaN,122.800003,131.929993,NaN,...,118.669998,14.870000,4.910000,119.050003,NaN,158.229996,112.500000,287.570007,50.439999,163.500000


In [80]:
df.isna().all().any()

np.True_

In [81]:
# Dropping any stocks where we have completely null data
df.drop(df.columns[df.isna().all()], axis=1, inplace=True)
df.shape

(2580, 614)

## Accounting for Survivorship Bias

### Test nulling data for a single removed stock

In [83]:
removed

,Ticker,Security
Date,,
2015-01-27,COV,Covidien
2015-01-27,SWY,Safeway
2015-03-12,PETM,PetSmart
2015-03-18,CFN,Carefusion
2015-03-23,AGN,Allergan
...,...,...
2024-12-23,QRVO,Qorvo
2025-03-24,FMC,FMC Corporation
2025-03-24,CE,Celanese


In [84]:
removed[removed.Ticker == 'QRVO']

,Ticker,Security
Date,,
2024-12-23,QRVO,Qorvo


In [85]:
removed[removed.Ticker == 'QRVO'].index[0]

Timestamp('2024-12-23 00:00:00')

In [86]:
df['QRVO']

Date
2015-01-02    70.400002
2015-01-05    67.629997
2015-01-06    64.669998
2015-01-07    66.650002
2015-01-08    67.690002
                ...    
2025-03-31    72.410004
2025-04-01    72.230003
2025-04-02    72.639999
2025-04-03    61.090000
2025-04-04    56.349998
Name: QRVO, Length: 2580, dtype: float64

In [87]:
pd.date_range(start, removed[removed.Ticker == 'QRVO'].index[0], freq='B')

DatetimeIndex(['2015-01-01', '2015-01-02', '2015-01-05', '2015-01-06',
               '2015-01-07', '2015-01-08', '2015-01-09', '2015-01-12',
               '2015-01-13', '2015-01-14',
               ...
               '2024-12-10', '2024-12-11', '2024-12-12', '2024-12-13',
               '2024-12-16', '2024-12-17', '2024-12-18', '2024-12-19',
               '2024-12-20', '2024-12-23'],
              dtype='datetime64[ns]', length=2603, freq='B')

In [88]:
df['QRVO'] = df['QRVO'].reindex(pd.date_range(start, removed[removed.Ticker == 'QRVO'].index[0], freq='B'))
df['QRVO']

Date
2015-01-02    70.400002
2015-01-05    67.629997
2015-01-06    64.669998
2015-01-07    66.650002
2015-01-08    67.690002
                ...    
2025-03-31          NaN
2025-04-01          NaN
2025-04-02          NaN
2025-04-03          NaN
2025-04-04          NaN
Name: QRVO, Length: 2580, dtype: float64

In [90]:
# Everything before the removal date is populated
df['QRVO'].loc[:removed[removed.Ticker == 'QRVO'].index[0]]

Date
2015-01-02    70.400002
2015-01-05    67.629997
2015-01-06    64.669998
2015-01-07    66.650002
2015-01-08    67.690002
                ...    
2024-12-17    70.949997
2024-12-18    68.500000
2024-12-19    68.800003
2024-12-20    70.849998
2024-12-23    71.540001
Name: QRVO, Length: 2511, dtype: float64

In [91]:
# Everything after the removal date has been nulled
df['QRVO'].loc[removed[removed.Ticker == 'QRVO'].index[0]:]

Date
2024-12-23    71.540001
2024-12-24          NaN
2024-12-26          NaN
2024-12-27          NaN
2024-12-30          NaN
                ...    
2025-03-31          NaN
2025-04-01          NaN
2025-04-02          NaN
2025-04-03          NaN
2025-04-04          NaN
Name: QRVO, Length: 70, dtype: float64

### Test nulling data for a single added stock

In [93]:
added

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
Date added,,,,,,,,
2015-01-27,HCA,HCA Healthcare,Health Care,Health Care Facilities,"Nashville, Tennessee",2015-01-27,860730,1968
2015-03-12,SWKS,Skyworks Solutions,Information Technology,Semiconductors,"Irvine, California",2015-03-12,4127,2002
2015-03-17,HSIC,Henry Schein,Health Care,Health Care Distributors,"Melville, New York",2015-03-17,1000228,1932
2015-03-20,EQIX,Equinix,Real Estate,Data Center REITs,"Redwood City, California",2015-03-20,1101239,1998
2015-04-07,O,Realty Income,Real Estate,Retail REITs,"San Diego, California",2015-04-07,726728,1969
...,...,...,...,...,...,...,...,...
2024-12-23,WDAY,"Workday, Inc.",Information Technology,Application Software,"Pleasanton, California",2024-12-23,1327811,2005
2025-03-24,TKO,TKO Group Holdings,Communication Services,Movies & Entertainment,"New York City, New York",2025-03-24,1973266,2023
2025-03-24,DASH,DoorDash,Consumer Discretionary,Specialized Consumer Services,"San Francisco, California",2025-03-24,1792789,2012


In [96]:
added[added.Symbol=='AOS'].index[0]

Timestamp('2017-07-26 00:00:00')

In [97]:
pd.date_range(added[added.Symbol=='AOS'].index[0], end, freq='B')

DatetimeIndex(['2017-07-26', '2017-07-27', '2017-07-28', '2017-07-31',
               '2017-08-01', '2017-08-02', '2017-08-03', '2017-08-04',
               '2017-08-07', '2017-08-08',
               ...
               '2025-03-24', '2025-03-25', '2025-03-26', '2025-03-27',
               '2025-03-28', '2025-03-31', '2025-04-01', '2025-04-02',
               '2025-04-03', '2025-04-04'],
              dtype='datetime64[ns]', length=2008, freq='B')

In [98]:
df['AOS'] = df['AOS'].reindex(pd.date_range(added[added.Symbol=='AOS'].index[0], end, freq='B'))
df['AOS']

Date
2015-01-02          NaN
2015-01-05          NaN
2015-01-06          NaN
2015-01-07          NaN
2015-01-08          NaN
                ...    
2025-03-31    65.360001
2025-04-01    65.599998
2025-04-02    66.120003
2025-04-03    63.660000
2025-04-04    61.990002
Name: AOS, Length: 2580, dtype: float64

In [99]:
# Everything before the added date is null
df['AOS'].loc[:added[added.Symbol=='AOS'].index[0]]

Date
2015-01-02          NaN
2015-01-05          NaN
2015-01-06          NaN
2015-01-07          NaN
2015-01-08          NaN
                ...    
2017-07-20          NaN
2017-07-21          NaN
2017-07-24          NaN
2017-07-25          NaN
2017-07-26    47.457062
Name: AOS, Length: 646, dtype: float64

In [100]:
# Everything before the added date is populated
df['AOS'].loc[added[added.Symbol=='AOS'].index[0]:]

Date
2017-07-26    47.457062
2017-07-27    47.334145
2017-07-28    47.667805
2017-07-31    47.018066
2017-08-01    47.184887
                ...    
2025-03-31    65.360001
2025-04-01    65.599998
2025-04-02    66.120003
2025-04-03    63.660000
2025-04-04    61.990002
Name: AOS, Length: 1935, dtype: float64

## Creating functions to account for

In [101]:
def pricefilter_removed(ticker):
    if ticker in df.columns:
        df[ticker] = df[ticker].reindex(pd.date_range(start, removed[removed.Ticker==ticker].index[0], freq='B'))

def pricefilter_added(ticker):
    if ticker in df.columns:
        df[ticker] = df[ticker].reindex(pd.date_range(added[added.Symbol==ticker].index[0], end, freq='B'))

In [102]:
for ticker in removed.Ticker:
    pricefilter_removed(ticker)

In [103]:
for ticker in added.Symbol:
    pricefilter_added(ticker)

In [104]:
df

,A,AA,AAL,AAP,AAPL,ABBV,ABNB,ABT,ACE,ACGL,...,XEL,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,37.273823,35.912807,51.079914,141.844131,24.320438,43.156208,NaN,36.915031,NaN,NaN,...,26.153721,59.510231,46.345871,19.412529,33.441803,42.833298,100.714760,NaN,21.878542,40.157536
2015-01-05,36.575386,33.832211,51.051487,139.974518,23.635283,42.344051,NaN,36.923233,NaN,NaN,...,25.856850,57.881908,46.024578,18.988977,31.360487,41.963024,104.471764,NaN,21.058773,39.916458
2015-01-06,36.005630,34.080982,50.255596,139.876099,23.637514,42.134453,NaN,36.503952,NaN,NaN,...,25.994421,57.574200,45.453403,18.734856,31.176060,41.447956,103.586182,NaN,20.254478,39.527031
2015-01-07,36.483501,34.962975,50.227161,142.881866,23.968960,43.837383,NaN,36.799927,NaN,NaN,...,26.218882,58.157578,46.631454,18.974867,31.421959,42.821465,106.162430,NaN,20.447815,40.342964
2015-01-08,37.577080,35.958031,50.843040,144.134262,24.889906,44.295864,NaN,37.556309,NaN,NaN,...,26.523003,59.125603,47.782749,19.497240,31.659069,43.567413,107.289520,NaN,20.733965,40.964191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-31,116.732002,NaN,NaN,NaN,222.130005,209.520004,119.459999,132.649994,NaN,96.180000,...,70.790001,118.930000,NaN,NaN,119.459999,157.360001,113.180000,282.559998,NaN,164.649994
2025-04-01,114.050003,NaN,NaN,NaN,223.190002,206.270004,121.180000,131.690002,NaN,96.139999,...,70.720001,119.040001,NaN,NaN,119.879997,158.919998,112.370003,283.070007,NaN,161.910004
2025-04-02,115.989998,NaN,NaN,NaN,223.889999,205.190002,122.800003,131.929993,NaN,96.820000,...,70.900002,118.669998,NaN,NaN,119.050003,158.229996,112.500000,287.570007,NaN,163.500000


## Calculate Returns

In [106]:
ret = df.pct_change()
ret

/tmp/ipykernel_34795/948626315.py:1: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ret = df.pct_change()


,A,AA,AAL,AAP,AAPL,ABBV,ABNB,ABT,ACE,ACGL,...,XEL,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05,-0.018738,-0.057935,-0.000557,-0.013181,-0.028172,-0.018819,NaN,0.000222,NaN,NaN,...,-0.011351,-0.027362,-0.006933,-0.021818,-0.062237,-0.020318,0.037303,NaN,-0.037469,-0.006003
2015-01-06,-0.015578,0.007353,-0.015590,-0.000703,0.000094,-0.004950,NaN,-0.011355,NaN,NaN,...,0.005320,-0.005316,-0.012410,-0.013383,-0.005881,-0.012274,-0.008477,NaN,-0.038193,-0.009756
2015-01-07,0.013272,0.025879,-0.000566,0.021489,0.014022,0.040417,NaN,0.008108,NaN,NaN,...,0.008635,0.010133,0.025918,0.012811,0.007887,0.033138,0.024871,NaN,0.009545,0.020642
2015-01-08,0.029975,0.028460,0.012262,0.008765,0.038422,0.010459,NaN,0.020554,NaN,NaN,...,0.011599,0.016645,0.024689,0.027530,0.007546,0.017420,0.010617,NaN,0.013994,0.015399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-31,0.002485,0.000000,0.000000,0.000000,0.019413,0.020605,-0.010191,0.013989,NaN,0.011782,...,0.015347,0.010193,0.000000,0.000000,0.000167,0.010272,0.010355,0.006304,0.000000,0.009441
2025-04-01,-0.022976,0.000000,0.000000,0.000000,0.004772,-0.015512,0.014398,-0.007237,NaN,-0.000416,...,-0.000989,0.000925,0.000000,0.000000,0.003516,0.009914,-0.007157,0.001805,0.000000,-0.016641
2025-04-02,0.017010,0.000000,0.000000,0.000000,0.003136,-0.005236,0.013369,0.001822,NaN,0.007073,...,0.002545,-0.003108,0.000000,0.000000,-0.006924,-0.004342,0.001157,0.015897,0.000000,0.009820


In [108]:
df.columns[df.isna().all()]

Index(['ACE', 'ADT', 'ALTR', 'BHI', 'CA', 'CCE', 'DD', 'DNB', 'DOW', 'EMC',
       'EQT', 'FSLR', 'INFO', 'NE', 'PCG', 'PCP', 'PETM', 'PLL', 'SBNY', 'SE',
       'SNDK', 'STI', 'TE', 'TEG'],
      dtype='object')

In [107]:
ret.columns[ret.isna().all()]

Index(['ACE', 'ADT', 'ALTR', 'BHI', 'CA', 'CCE', 'DD', 'DNB', 'DOW', 'EMC',
       'EQT', 'FSLR', 'INFO', 'NE', 'PCG', 'PCP', 'PETM', 'PLL', 'SBNY', 'SE',
       'SNDK', 'STI', 'TE', 'TEG'],
      dtype='object')

In [114]:
mth_ret = (ret + 1).resample('M').agg(pd.Series.prod, skipna=False)
mth_ret

/tmp/ipykernel_34795/1841358604.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  mth_ret = (ret + 1).resample('M').agg(pd.Series.prod, skipna=False)


,A,AA,AAL,AAP,AAPL,ABBV,ABNB,ABT,ACE,ACGL,...,XEL,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-28,1.117553,0.946758,0.977935,0.974403,1.100777,1.002486,NaN,1.058311,NaN,NaN,...,0.940048,1.020435,1.059564,1.036447,1.051083,1.122164,1.073952,NaN,1.117725,1.078633
2015-03-31,0.986788,0.873563,1.101879,0.966565,0.968628,0.967603,NaN,0.978045,NaN,NaN,...,0.995929,0.960018,0.961345,0.946540,0.980952,0.970534,0.977964,NaN,1.009723,1.004339
2015-04-30,0.995668,1.038699,0.916806,0.955308,1.005786,1.113705,NaN,1.007068,NaN,NaN,...,0.974146,1.027882,1.002162,0.894942,1.057110,1.097720,0.934650,NaN,1.049629,0.961325
2015-05-31,0.995649,0.933490,0.877407,1.071469,1.045339,1.029848,NaN,1.046962,NaN,NaN,...,1.004128,0.983229,1.020392,0.993043,0.991624,1.048278,1.040732,NaN,1.021155,1.120441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31,0.975490,1.000000,1.000000,1.000000,1.055155,0.971410,0.965469,0.952345,NaN,0.916898,...,0.930540,0.911919,1.000000,1.000000,0.915345,0.970276,0.944405,0.948943,1.000000,0.929700
2025-01-31,1.127885,1.000000,1.000000,1.000000,0.942417,1.044649,0.998174,1.136958,NaN,1.007796,...,1.003507,0.993121,1.000000,1.000000,1.069126,0.972719,1.036448,1.014810,1.000000,1.052088
2025-02-28,0.844245,1.000000,1.000000,1.000000,1.025872,1.136650,1.058702,1.078793,NaN,0.998281,...,1.072917,1.051444,1.000000,1.000000,1.058497,1.203900,0.952868,0.803822,1.000000,0.978584


In [115]:
rolling = mth_ret.rolling(12).apply(np.prod)#.dropna(how='all')
rolling

,A,AA,AAL,AAP,AAPL,ABBV,ABNB,ABT,ACE,ACGL,...,XEL,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31,0.972971,1.0,0.799127,1.0,1.307053,1.188590,0.965256,1.048126,NaN,1.307643,...,1.123209,1.112616,0.899342,1.0,1.025712,1.047221,0.875422,1.413017,0.915168,0.833681
2025-01-31,1.172724,1.0,0.771609,1.0,1.286097,1.159362,0.910018,1.153664,NaN,1.187381,...,1.165554,1.074547,0.921081,1.0,1.115341,1.027856,0.879156,1.636151,0.958196,0.919667
2025-02-28,0.937737,1.0,0.700255,1.0,1.344298,1.230585,0.881882,1.186973,NaN,1.115511,...,1.420952,1.101108,0.979424,1.0,1.041786,1.151839,0.846073,1.127272,1.007608,0.852234


In [ ]:
top = rolling.iloc[0].nlargest(5)
top

In [ ]:
top.name

In [ ]:
invested = mth_ret[top.name:].iloc[1][top.index]
invested

In [ ]:
invested.mean()

In [ ]:
def top_performers(date):
    top = rolling.loc[date].nlargest(5)
    invested = mth_ret[top.name:].iloc[1][top.index]

    return invested.mean()

In [ ]:
top_performers(rolling.iloc[0].name)

In [ ]:
initial_inv = 100

for date in rolling.index[:-1]:
    
    initial_inv *= top_performers(date)

print(initial_inv)

In [ ]:
sp500 = yf.download(tickers=['^GSPC'], start=start, end=end)['Close']
sp500

In [ ]:
sp500_ret = sp500.pct_change().dropna() + 1
sp500_ret

In [ ]:
sp500_ret = sp500_ret.resample('M').prod()
sp500_ret

In [ ]:
initial_inv = 100

for date in rolling.index[:-1]:
    
    # print(sp500_ret.loc[date])
    initial_inv *= sp500_ret.loc[date]

initial_inv



In [ ]:
sp500_ret.mean()

In [ ]:
100*1.24**10